In [1]:
import ee

# 第一步：运行 Authenticate，会弹出一个网址
ee.Authenticate()

# 第二步：在弹出的网页里选择你的 Google 账号（cding53@wisc.edu），
# 授权后会得到一串 Authorization code。

# 第三步：复制这个 Authorization code，粘贴回 Notebook 里
# 然后运行 Initialize 完成连接。
ee.Initialize()

# 测试一下是否成功
print(ee.Image("CGIAR/SRTM90_V4").getInfo()["id"])


Enter verification code:  4/1AVMBsJjIYk5l64gOY-BbHg4Wcr7lgOdRS---JHW5sUmpFMEhmYx4yhCKAT0



Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


CGIAR/SRTM90_V4


In [2]:
ee.Initialize()

In [2]:
#!/usr/bin/env python3
import os
import re
import pandas as pd
import numpy as np
import ee
from tqdm import tqdm

# ---------------- EE auth ----------------
ee.Authenticate()
ee.Initialize()

# ---------------- Paths ------------------
input_csv  = r"/mnt/cephfs-mount/chenchen/CygnssDataCsvLand/2018_08/20180801.csv"
output_csv = r"/mnt/cephfs-mount/chenchen/water_body_fraction/2018_08/20180801.csv"

# ------------- Helpers -------------------
def date_from_csv_path(path):
    """Parse YYYYMMDD from filename like 20180801.csv -> '2018-08-01'."""
    base = os.path.basename(path)
    m = re.match(r"(\d{8})\.csv$", base)
    if not m:
        raise ValueError(f"Cannot parse date from filename: {base}")
    ymd = m.group(1)
    return f"{ymd[:4]}-{ymd[4:6]}-{ymd[6:8]}"

def calculate_water_fraction(region, date_ee):
    """Return fraction of water in region on the given date (EE Date)."""
    # Load Sentinel-1 IW VV on that day
    sentinel1 = (ee.ImageCollection("COPERNICUS/S1_GRD")
                 .filterBounds(region)
                 .filterDate(date_ee, date_ee.advance(1, 'day'))
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')))

    # No data for day/region
    try:
        if sentinel1.size().getInfo() == 0:
            return None
    except Exception:
        return None

    vv = sentinel1.mosaic().select('VV')

    # Simple threshold (dB) for debug; adjust as needed
    water_mask = vv.lt(-17)

    water_area = (water_mask.multiply(ee.Image.pixelArea())
                  .reduceRegion(
                      reducer=ee.Reducer.sum(),
                      geometry=region,
                      scale=10,
                      maxPixels=1e13
                  ).get("VV"))

    try:
        water_m2 = water_area.getInfo()
        total_m2 = region.area().getInfo()
    except Exception:
        return None

    if water_m2 is None or not total_m2:
        return None

    return float(water_m2) / float(total_m2)

# ------------- Main ----------------------
def main():
    # 1) Date from filename (applies to all rows)
    date_str = date_from_csv_path(input_csv)
    date_ee  = ee.Date(date_str)
    tqdm.write(f"Using date from filename: {date_str}")

    # 2) Read CSV
    data = pd.read_csv(input_csv)

    # 3) Ensure output dir
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)

    results = []

    # 4) Loop rows; build polygon from c1..c4 and compute fraction
    for _, row in tqdm(data.iterrows(), total=len(data), desc="Processing rows"):
        try:
            region_coords = [
                [float(row['c1_lon']), float(row['c1_lat'])],
                [float(row['c2_lon']), float(row['c2_lat'])],
                [float(row['c3_lon']), float(row['c3_lat'])],
                [float(row['c4_lon']), float(row['c4_lat'])],
                [float(row['c1_lon']), float(row['c1_lat'])],  # close
            ]
        except (KeyError, TypeError, ValueError):
            continue

        region = ee.Geometry.Polygon([region_coords], None, False)

        frac = calculate_water_fraction(region, date_ee)
        if frac is not None and np.isfinite(frac):
            tqdm.write(f"{date_str} frac={frac:.4f} @ {region_coords[0]}")
            results.append({"water_fraction": float(frac)})

    # 5) Save
    pd.DataFrame(results).to_csv(output_csv, index=False)
    print(f"Water body fractions saved to {output_csv}")

if __name__ == "__main__":
    main()

Using date from filename: 2018-08-01


Processing rows: 100%|████████████████| 550115/550115 [27:49:33<00:00,  5.49it/s]

Water body fractions saved to /mnt/cephfs-mount/chenchen/water_body_fraction/2018_08/20180801.csv
